<hr size=7 color=#8D84B5 > </hr> 

<div align="center">

# <font color = #6b4cde face="Verdana"> **Universities and Gentrification**
## <font color = #6b4cde face="Verdana"> **UMD CMSC320 Data Science, Spring 2023** </font>
## <font color = #6b4cde face="Verdana"> **Joe Diaz and Connor Pymm** </font>
</center>

</div>

<hr size=7 color=#8D84B5 > </hr> 

### 🙏RUN ME FIRST🙏

In [ ]:
%pip install plotly
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

<hr size=7 color=#8D84B5 > </hr> 

<div align="center">

## <font color = #6b4cde face="Verdana"> **Data Curation** </font>
</center>

</div>

<hr size=7 color=#8D84B5 > </hr> 

### Selecting Datasets

In order to perform analysis on colleges and their surrounding regions, we needed to
find some subset of colleges, a dataset with characteristics of those colleges on a yearly basis, 
and then a dataset with characteristics of their nearby geographical areas. again yearly. 

Initially, we decided to limit our analysis to the top 100 universities in the country 
according to current US News rankings, under the assumption that more highly ranked universities 
might have a more significant impact on their respective communities. We used Andy Reiter's
“U.S. News & World Report Historical Liberal Arts College and University Rankings” dataset (**citation**).
  
In order to obtain college characteristics, we discovered that the Department of
Education has extensive data available on accredited universities called the College Scorecard, which has
a public API for programatically querying data.
  
In order to obtain characteristics of the region around each university, we needed a dataset that would contain
demographic and economic data for defined geographical regions associated with the location of the University.
We found that the American Community Survey yearly data from the Census had the housing cost and income data we
wanted to analyze, and that its Public Use Microdata API from the census allowed us to programatically request that
data for geographical groupings called "Public Use Microdata Areas," which are the smallest geographical entities that
the Census collects yearly data from.

### Extract, Transform, and Load

Since we queried a *substantial* amount of data from *ridiculously large* datasets,
and requesting federal data from the Department of Education and the Census required
registration for and usage of API keys, we decided that on top of the source datasets that
we were able to download in full, stored in our repository under ETL/source_data, we would
create modules for making federal API requests and loading the results into csv files for usage
later. 
  
Dataframes that we generated from data that we queried were stored under ETL/generated_data
as csv, and then loaded into the notebook when needed, specifically: we built ScorecardData.csv using
our scorecard_client.py module, which defines a CollegeScorecardClient object that can be used to query
DoE data, given a valid API key, set of desired variables, and set of colleges using IPEDS IDs, we built 
college_FIPs by combining the university list we got from Reiter with state FIPs data from DoE and county 
FIPs data by collecting them manually university by university.

For the rest of this tutorial, we will be using the data we collected by default, but if you would like to
recreate the analysis of this tutorial using a different set of colleges, and thus your own datasets, you can
fork this repository and use the modules provided in the ETL/ directory to do so.

<hr size=7 color=#8D84B5 > </hr> 

<div align="center">

## <font color = #6b4cde face="Verdana"> **Data Processing** </font>
</center>

</div>

<hr size=7 color=#8D84B5 > </hr> 

### Loading and Representation

Here, we load the data we have downloaded or generated locally into our
notebook for use to use in our analysis. We stored each of our datasets as
csv, so they are easily loaded into Pandas Dataframes. We will also have to define a few new terms so that the reader can understand what we are talking about:
1. **MSA**: The MSA is the metropolitan statistical area in the U.S. Census. It includes 384 census-designated regions in the U.S. with more than 50,000 people in each block. We use this data as a standard for the general region around colleges and as a sort of control variable compared to our more limited and specific PUMA.
2. **PUMA**: The PUMAs stands for "Public Use Microdata Areas", which is kind of in between a county and a school system in the Census. IT was the smallest readily available geography in the Census dataset that we could convert to using zip codes and tracts, so we use it as our specific locality that we are comparing against our control (MSA). If our hypothesis is correct, these should be more volatile since the prices of colleges should be driving up prices in these localities.
3. **Tract**: A Census Tract is a specific section of land, even more specific than the PUMA, that we use to connect the zip codes from the top 100 universities and the MSAs and PUMA's from the Census. They are very finnicky and hard to work with, but we found a U.S. government API that allows us to convert between Zip Codes, Census Tracts, and MSAs, which allowed us to complete our analysis.
4. **Zip Code**: You likely already know what a zip code is, but working with Zip Codes was a very difficult challenge in this project since they are maintained by the U.S. Postal Service, not the Census, so neither agency has a standard measurement for localities, hence why we needed to convert from Zip -> Tract -> MSA and PUMA.

In [ ]:
# Read dataframes from Scorecard generated data
scard_df = pd.read_csv("ETL/generated_data/ScorecardData.csv")
fips_df = pd.read_csv("ETL/generated_data/college_FIPs.csv")
cpi_df = pd.read_csv("ETL/source_data/cpi_all.csv").groupby("Year")["Value"].mean()
scard_df.head()

In [ ]:
years = range(2009, 2020)
msa_path_format = "ETL/generated_data/MSA{y}.csv"
MSA_frames = [
    pd.read_csv(msa_path_format.format(y=yr)).assign(year=yr)
    for yr in years
]
msa_df = pd.concat(MSA_frames)
msa_df.columns = ["name", "msa_income", "msa_rent", "msa", "year"]
print(msa_df["name"].unique().shape)
msa_df

In [ ]:
years = range(2009, 2020)
puma_path_format = "ETL/generated_data/PUMA{y}.csv"
PUMA_frames = [
    pd.read_csv(puma_path_format.format(y=yr)).assign(year=yr)
    for yr in years
]
puma_df = pd.concat(PUMA_frames).reset_index(drop=True)
puma_df.columns = ["puma_income", "puma_rent", "state", "puma", "year"]
puma_df["state"] = puma_df["state"].astype("int")
puma_df["puma"] = puma_df["puma"].astype("int")
puma_df["year"] = puma_df["year"].astype("int")

print(puma_df["puma"].unique().shape)
puma_df

### Data Cleaning and Reshaping

The data that we have still uses the variable names and formatting of our
original sources, and those variable names are unweildy and not ideal for usage
in analysis later, so we rename our columns to be more human readable and
developer friendly. Additionally, cost data in our sources does not account for
inflation, so we should use an all-consumers/all-goods CPI to transform our dollar
values to a standard value.

In [ ]:
# Rename columns to be more readable, usable
scard_df = scard_df.rename(
    columns={
        "student.size": "size",
        "cost.tuition.in_state": "in_state_tuition",
        "cost.tuition.out_of_state": "out_state_tuition",
        "cost.avg_net_price.public": "public_net_price",
        "cost.avg_net_price.private": "private_net_price",
        "id": "id",
        "school.name": "name",
        "school.carnegie_size_setting": "size_setting",
        "school.zip": "zip",
        "school.state_fips": "state_fips",
        "school.region_id": "region_id",
        "school.locale": "locale",
        "school.ownership": "ownership"
    }
)

# Join county FIPs codes into College Scorecard dataframe for use later in
# associating with Census geographies.
scard_df = pd.merge(
    scard_df, 
    fips_df[["id", "county", "cbsa", "puma"]], 
    on="id", how="left").drop_duplicates()
print(scard_df)

# Combine public and private net prices into a single net price column, and drop those columns
scard_df["net_cost"] = scard_df.apply(lambda row: 
            row["public_net_price"] if (row["ownership"] == 1) else row["private_net_price"],
        axis=1
)
scard_df["net_cost_adjusted"] = scard_df.apply(lambda row: 
            (row["net_cost"]/cpi_df.at[row["year"]]) * 100,
        axis=1
)
scard_df["in_tuition_adjusted"] = scard_df.apply(lambda row: 
            (row["in_state_tuition"]/cpi_df.at[row["year"]]) * 100,
        axis=1
)
scard_df["out_tuition_adjusted"] = scard_df.apply(lambda row: 
            (row["out_state_tuition"]/cpi_df.at[row["year"]]) * 100,
        axis=1
)
scard_df["non_tuition_expenses_adj"] = scard_df.apply(lambda row: 
            row["net_cost_adjusted"] - row["in_tuition_adjusted"],
        axis=1
)



scard_df["state_fips"] = scard_df["state_fips"].astype(int)
scard_df["year"] = scard_df["year"].astype(int)
scard_df.drop(["public_net_price", "private_net_price"], axis=1, inplace=True)
scard_df.head()

In [ ]:
locale_map = {
    11:	"City: Large",
    12:	"City: Midsize",
    13:	"City: Small",
    21:	"Suburb: Large",
    22:	"Suburb: Midsize",
    23:	"Suburb: Small",
    31:	"Town: Fringe",
    32:	"Town: Distant",
    33:	"Town: Remote",
    41:	"Rural: Fringe",
    42:	"Rural: Distant",
    43:	"Rural: Remote"
}
ownership_map = {
    1:	"Public",
    2:	"Private nonprofit",
    3:	"Private for-profit"
}
region_map = {
    0:	"U.S. Service Schools",
    1:	"New England",
    2:	"Mid East",
    3:	"Great Lakes",
    4:	"Plains",
    5:	"Southeast",
    6:	"Southwest",
    7:	"Rocky Mountains",
    8:	"Far West",
    9:	"Outlying Areas"
}

scard_df["region"] = scard_df.apply(lambda row: 
            region_map[row["region_id"]],
        axis=1
)
scard_df["ownership"] = scard_df.apply(lambda row: 
            ownership_map[row["ownership"]],
        axis=1
)
scard_df["locale"] = scard_df.apply(lambda row: 
            locale_map[row["locale"]],
        axis=1
)

We can note that some rows do not have cost data associated with them, thus they are missing data.
Since we will use this cost data later in our analysis, we need to either interpolate the missing data
or drop the invalid rows. Here, we experiment with dropping rows with missing data.

In [ ]:
scard_clipped = scard_df.dropna(subset=["net_cost", "in_state_tuition", "out_state_tuition"]).copy()
#print(scard_clipped["name"].unique().shape)
#print(scard_clipped.to_string())

It seems as if the clipped dataframe after dropping null cost data is just the data after 2009.
To verify that this is true, I try querying the original dataset purely by restricting the years.
If there is complete cost data from 2009 to 2020, then the resulting dataframe should be equal to the
dataframe resulting from dropping null data. Run the next code cell to confirm this.

In [ ]:
scard_clipped_year = scard_df[scard_df["year"] >= 2009].copy()
scard_clipped_year.equals(scard_clipped)

In [ ]:
msa_df["msa"] = msa_df["msa"].str.replace("]", "").astype(int)
msa_df

In [ ]:
scard_clipped_geo = scard_clipped.dropna(subset=["cbsa", "puma"]).copy()
scard_clipped_geo

In [ ]:
scard_clipped_geo["puma"] = scard_clipped_geo["puma"].astype(int)
scard_clipped_geo["cbsa"] = scard_clipped_geo["cbsa"].astype(int)
print(scard_clipped_geo["puma"].unique().shape)
scard_clipped_geo

In [ ]:
scard_df = scard_clipped_geo
scard_df = pd.merge(scard_df, puma_df, left_on=["puma", "year"], right_on=["puma", "year"], how="inner")
scard_df["name"].unique().shape

In [ ]:
scard_df = pd.merge(scard_df, msa_df, left_on=["cbsa", "year"], right_on=["msa", "year"], how="left")
scard_df["name_y"].unique().shape

<hr size=7 color=#8D84B5 > </hr> 

<div align="center">

## <font color = #6b4cde face="Verdana"> **Exploratory Analysis and Data Visualization** </font>
</center>

</div>

<hr size=7 color=#8D84B5 > </hr> 

In this section, we will be analyzing different components of our dataset graphically to display relationships present within it. For example, in this next block, we look at a violin plot of the net cost of every university and how it changes every year. Since the purpose of a violin plot is to visualize a distribute of data, we are able to see the increasing volatility in the net cost of university as time goes on. 

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Violin(
        x=scard_df['year'], 
        y=scard_df['net_cost'],
        box_visible=True,
        meanline_visible=True
    )
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Violin(
        x=scard_df['year'], 
        y=scard_df['in_state_tuition'],
        box_visible=True,
        meanline_visible=True
    )
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Violin(
        x=scard_df['year'], 
        y=scard_df['out_state_tuition'],
        box_visible=True,
        meanline_visible=True
    )
)
fig.show()

In [ ]:
fig = px.scatter(scard_df, x="year", y="in_state_tuition", facet_col="ownership")
fig.show()


In [ ]:
fig = px.scatter(scard_df, x="year", y="in_tuition_adjusted", facet_col="ownership")
fig.show()

In [ ]:
fig = px.scatter(scard_df, x="year", y="out_tuition_adjusted", facet_col="ownership")
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=2)
fig.add_trace(
    go.Violin(x=msa_df["year"], 
              y=msa_df["msa_income"], 
              box_visible=True,
              meanline_visible=True
             ),
    row=1, col=1
)
fig.add_trace(
    go.Violin(x=msa_df["year"], 
              y=msa_df["msa_rent"], 
              box_visible=True,
              meanline_visible=True
             ),
    row=1, col=2
)

fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_xaxes(title_text="Year", row=1, col=2)
fig.update_yaxes(title_text="Median Household Income", row=1, col=1)
fig.update_yaxes(title_text="Median Monthly Rent ", row=1, col=2)
fig.update_layout(title_text="Household Income vs. Monthly Rent Over Time for MSA Adjusted for Inflation")
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=2)
fig.add_trace(
    go.Violin(x=puma_df["year"], 
              y=puma_df["puma_income"], 
              box_visible=True,
              meanline_visible=True
             ),
    row=1, col=1
)
fig.add_trace(
    go.Violin(x=puma_df["year"], 
              y=puma_df["puma_rent"], 
              box_visible=True,
              meanline_visible=True
             ),
    row=1, col=2
)

fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_xaxes(title_text="Year", row=1, col=2)
fig.update_yaxes(title_text="Median Household Income", row=1, col=1)
fig.update_yaxes(title_text="Median Monthly Rent", row=1, col=2)
fig.update_layout(title_text="Household Income vs. Monthly Rent Over Time for PUMA")
fig.show()

In [ ]:
msa_df["msa_income_adj"] = msa_df.apply(lambda row: 
            (row["msa_income"]/cpi_df.at[row["year"]]) * 100,
        axis=1
)
msa_df["msa_rent_adj"] = msa_df.apply(lambda row: 
            (row["msa_rent"]/cpi_df.at[row["year"]]) * 100,
        axis=1
)
scard_df["msa_income_adj"] = scard_df.apply(lambda row: 
            (row["msa_income"]/cpi_df.at[row["year"]]) * 100,
        axis=1
)
scard_df["msa_rent_adj"] = scard_df.apply(lambda row: 
            (row["msa_rent"]/cpi_df.at[row["year"]]) * 100,
        axis=1
)
scard_df["msa_rent_adj_ytd"] = scard_df.apply(lambda row: 
            (row["msa_rent"]/cpi_df.at[row["year"]]) * 100 * 12,
        axis=1
)


fig = make_subplots(rows=1, cols=2)
fig.add_trace(
    go.Violin(x=msa_df["year"], 
              y=msa_df["msa_income_adj"], 
              box_visible=True,
              meanline_visible=True
             ),
    row=1, col=1
)
fig.add_trace(
    go.Violin(x=msa_df["year"], 
              y=msa_df["msa_rent_adj"], 
              box_visible=True,
              meanline_visible=True
             ),
    row=1, col=2
)

fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_xaxes(title_text="Year", row=1, col=2)
fig.update_yaxes(title_text="Median Household Income", row=1, col=1)
fig.update_yaxes(title_text="Median Monthly Rent ", row=1, col=2)
fig.update_layout(title_text="Household Income vs. Monthly Rent Over Time for MSA Adjusted for Inflation")
fig.show()

In [ ]:
puma_df["puma_income_adj"] = puma_df.apply(lambda row: 
            (row["puma_income"]/cpi_df.at[row["year"]]) * 100,
        axis=1
)
puma_df["puma_rent_adj"] = puma_df.apply(lambda row: 
            (row["puma_rent"]/cpi_df.at[row["year"]]) * 100,
        axis=1
)
scard_df["puma_income_adj"] = scard_df.apply(lambda row: 
            (row["puma_income"]/cpi_df.at[row["year"]]) * 100,
        axis=1
)
scard_df["puma_rent_adj"] = scard_df.apply(lambda row: 
            (row["puma_rent"]/cpi_df.at[row["year"]]) * 100,
        axis=1
)
scard_df["puma_rent_adj_ytd"] = scard_df.apply(lambda row: 
            (row["puma_rent"]/cpi_df.at[row["year"]]) * 100 * 12,
        axis=1
)

fig = make_subplots(rows=1, cols=2)
fig.add_trace(
    go.Violin(x=puma_df["year"], 
              y=puma_df["puma_income_adj"], 
              box_visible=True,
              meanline_visible=True
             ),
    row=1, col=1
)
fig.add_trace(
    go.Violin(x=puma_df["year"], 
              y=puma_df["puma_rent_adj"], 
              box_visible=True,
              meanline_visible=True
             ),
    row=1, col=2
)

fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_xaxes(title_text="Year", row=1, col=2)
fig.update_yaxes(title_text="Median Household Income", row=1, col=1)
fig.update_yaxes(title_text="Median Monthly Rent", row=1, col=2)
fig.update_layout(title_text="Household Income vs. Monthly Rent Over Time for PUMA Adjusted for Inflation")
fig.show()

In [ ]:
fig = px.scatter(scard_df, x="net_cost_adjusted", y="msa_rent_adj_ytd", trendline="ols", facet_col="ownership")
fig.show()

In [ ]:
fig = px.violin(scard_df, x="ownership", y="puma_rent_adj_ytd")
fig.show()

In [ ]:
fig = px.violin(scard_df, x="region", y="puma_rent_adj_ytd")
fig.show()

In [ ]:
fig = px.violin(scard_df, x="locale", y="puma_rent_adj_ytd")
fig.show()

<hr size=7 color=#8D84B5 > </hr> 

<div align="center">

## <font color = #6b4cde face="Verdana"> **Analysis, Hypothesis Testing, and Machine Learning** </font>
</center>

</div>

<hr size=7 color=#8D84B5 > </hr> 

Since monthly rent appears to increase linearly proportionately to household income, I will be using a Least Squares Linear Regression to highlight this relationship.

In [ ]:
scard_df["locale"].unique().shape

In [ ]:
fig = px.scatter(msa_df, x="msa_rent_adj", y="msa_income_adj", trendline="ols")
fig.show()

In [ ]:
results = px.get_trendline_results(fig)
print(results.px_fit_results.iloc[0].summary())

In [ ]:
fig = px.scatter(puma_df, x="puma_rent_adj", y="puma_income_adj", trendline="ols")
fig.show()

In [ ]:
results = px.get_trendline_results(fig)
print(results.px_fit_results.iloc[0].summary())

As can be seen by the OLS Regression Summary, the MSA plot has a much more accurate regression with an R-squared of 0.814 compared to PUMA's 0.721, but this can be explained by PUMA having significantly more data points (PUMAs are more specific than MSAs) and the variance among the data is significantly wider (For example, the monthly rent for MSA does not even exceed 1000, but the montly rent for PUMA goes well above 1100). This, as well as many other factors, shows a strong tendency for universities to affect pricing much more on a small scale, such as a city level, than a county or metropolitan-center-wide scale.

In [ ]:
scard_df.columns

In [ ]:
puma_rent_model = smf.ols(formula="puma_rent_adj ~ year * ownership + in_tuition_adjusted", data=scard_df)
puma_rent_res = puma_rent_model.fit()

msa_rent_model = smf.ols(formula="msa_rent_adj ~ year * ownership + in_tuition_adjusted", data=scard_df)
msa_rent_res = msa_rent_model.fit()

prm2 = smf.ols(formula="puma_rent_adj ~ year * ownership", data=scard_df)
prm2_res = prm2.fit()

num = len(scard_df["year"].values)
df_samples = scard_df[["year", "ownership", "in_tuition_adjusted"]]
df_samples = sm.add_constant(df_samples)
sample_set = np.column_stack(
    (
        np.ones(num), 
        scard_df["year"].values, 
        scard_df["ownership"].values,
        scard_df["in_tuition_adjusted"].values,
    )
)

df_samples2 = scard_df[["year", "ownership"]]
df_samples2 = sm.add_constant(df_samples)
sample_set2 = np.column_stack(
    (
        np.ones(num), 
        scard_df["year"].values, 
        scard_df["ownership"].values,
    )
)
scard_df["puma_rent_fit"] = puma_rent_res.predict(df_samples)
scard_df["puma_rent_fit2"] = prm2_res.predict(df_samples)
scard_df["msa_rent_fit"] = msa_rent_res.predict(df_samples)

In [ ]:
fig = make_subplots(rows=2, cols=1)

fig.update_layout(height=800, width=800, title_text="Interaction Model Public Use MicroData Area Rent Distribution and Trend Lines")
for i,id in enumerate (scard_df["ownership"].unique()):
    sub_df = scard_df[scard_df["ownership"] == id].copy().sort_values(by="year")

    trend_df = sub_df.groupby("year")["puma_rent_fit"].mean()
    fig.add_trace(
        go.Violin(
            x=sub_df['year'], 
            y=sub_df['puma_rent_adj'],
            name=id,
        ),
        row=(i+1), col=1
    )
    fig.add_trace(
        go.Scatter(
            x=trend_df.index, 
            y=trend_df,
            name=id,
        ),
        row=(i+1), col=1
    )
fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=1)

fig.update_layout(height=800, width=800, title_text="Interaction Model Metropolitan Statistical Area Rent Distribution and Trend Lines")
for i,id in enumerate (scard_df["ownership"].unique()):
    sub_df = scard_df[scard_df["ownership"] == id].copy().sort_values(by="year")

    trend_df = sub_df.groupby("year")["msa_rent_fit"].mean()
    fig.add_trace(
        go.Violin(
            x=sub_df['year'], 
            y=sub_df['msa_rent_adj'],
            name=id,
        ),
        row=(i+1), col=1
    )
    fig.add_trace(
        go.Scatter(
            x=trend_df.index, 
            y=trend_df,
            name=id,
        ),
        row=(i+1), col=1
    )
fig.show()

In [ ]:
print(puma_rent_res.summary())
print(puma_rent_res.pvalues)


In [ ]:
print(prm2_res.summary())
print(prm2_res.pvalues)


In [ ]:
print(msa_rent_res.summary())
print(msa_rent_res.pvalues)


In [ ]:
scard_df.columns

<hr size=7 color=#8D84B5 > </hr> 

<div align="center">

## <font color = #6b4cde face="Verdana"> **Insight and Policy Decision** </font>
</center>

</div>

<hr size=7 color=#8D84B5 > </hr> 

Based on our observations throughout our project, we can conclude:
1. There is a correlation between median income and median housing costs
2. As time continues, both median income and median housing costs will likely continue rising due to inflation.
3. Localities near universities are more volatile and are often significantly more expensive than the housing further out from them, creating a heavily gentrified environment (We can see an example of this in College Park)

If you would like to learn more about the tools we used to make this:
* [Census API](https://www.census.gov/data/developers/guidance/api-user-guide.Overview.html#list-tab-2080675447)
* [USNews API](FILL IN)
* [College Scorecard API](https://collegescorecard.ed.gov/data/documentation/)
* [Zip To Tract API/Excel Doc](https://www.huduser.gov/portal/datasets/usps_crosswalk.html#data)